In [150]:
import matplotlib.pyplot as plt
import numpy as np
import casperfpga
import time
from lwa_f import snap2_fengine
from lwa_f import blocks
#from lwautils import lwa_arx 
import pandas as pd
from cr_functions import *
import struct
import math
from scipy import special as sp
from scipy import optimize as oz
from scipy import stats as st
from lwa_antpos import mapping
import os
from astropy.time import Time

fname_prefix='/home/ubuntu/kp/data/2024August12/'


## Connect to SNAP2 boards

In [3]:
brdnames=['snap01','snap02','snap03','snap04','snap05','snap06','snap07','snap08','snap09','snap10','snap11']

#connect through Jack's fengine etcd interface for everything that can go through etcd
snapbrds = [snap2_fengine.Snap2FengineEtcd(brdname).fpga for brdname in brdnames]  
fpgfile='/home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c.fpg'


2024-07-19 01:14:50,976 - lwa_f.blocks.block:snap01 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-07-19 01:14:50,980 - lwa_f.blocks.block:snap02 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-07-19 01:14:50,984 - lwa_f.blocks.block:snap03 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-07-19 01:14:50,986 - lwa_f.blocks.block:snap04 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-07-19 01:14:50,989 - lwa_f.blocks.block:snap05 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-07-19 01:14:51,014 - lwa_f.blocks.block:snap06 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-07-19 01:14:51,017 - lwa_f.blocks.block:snap07 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-07-19 01:14:51,020 - lwa_f.blocks.block:snap08 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-07-19 01:14:51,023 - lwa_f.blocks.block:snap09 - INFO - eth - Couldn't find Ethernet core. 

## Completely reset trigger on all boards to make sure there aren't already trigger and veto settings

In [22]:


veto_roles_array=np.zeros(64,dtype=int)
core_roles_array=np.zeros(64,dtype=int)
for p in range(11):
    snapbrd=snapbrds[p]
    setup_coincidencer(snapbrd,0,
                          0,
                          0,
                          0,
                          0,
                          0,
                          core_roles_array,
                          veto_roles_array)
    setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0)


In [232]:
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]

#send trigger to just one board and all will send packets
software_trigger(snapbrds[1],1)

AssertionError: board didn't reset to listen

In [221]:
software_trigger(snapbrds[4],1)

Packetizer formed 64 packets
Board successfully returned to listening state after readout
Readout took 137669 clock cycles
Packetizer never overflowed
Ethernet block almost-full counter incremented by 0


## Configure Trigger 

In [147]:
#configure trigger
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]


current_ok_vetos=np.load('/home/ubuntu/kp/ok_vetos-2024-7-14.npy')
current_ok_core=np.load('/home/ubuntu/kp/ok_core-2024-7-14.npy')


coresnaps=[2,3,4,5,6,7,8,9,10] #zero indexed
#set up coincidencer
trigger_power_thresh=275**2
veto_power_thresh=175**2
trigger_window=int(round(1.96e8*100/3e8,1)) #number of clock cycles to travel 100 meters
veto_window=int(round(1.96e8*2000/3e8,1)) #number of clock cycles to travel 2000 meters
antenna_number_thresh=7
veto_number_thresh=2

bufferwait=2000
tries=15
#sparse array snaps need a longer trigger window
for b in coresnaps:
    snapbrd=snapbrds[b]
    veto_roles_array=current_ok_vetos[b,:]
    core_roles_array=current_ok_core[b,:]
    for attempt in range(tries):
        try:
            setup_coincidencer(snapbrd,trigger_power_thresh,
                          veto_power_thresh,
                          trigger_window,
                          veto_window,
                          antenna_number_thresh,
                          veto_number_thresh,
                          core_roles_array,
                          veto_roles_array)
            setvalue(snapbrd,"delay_trigger","cr_registers.xlsx",bufferwait)
        except:
                if attempt<tries-1:
                    continue
                else:
                    raise
        break
            
sparsesnaps=[0,1] #zero indexed
trigger_window=int(round(1.96e8*100/3e8,1)) #this time, same as the others

for b in sparsesnaps:
    snapbrd=snapbrds[b]
    veto_roles_array=current_ok_vetos[b,:]
    core_roles_array=current_ok_core[b,:]
    for attempt in range(tries):
        try:
            setup_coincidencer(snapbrd,trigger_power_thresh,
                          veto_power_thresh,
                          trigger_window,
                          veto_window,
                          antenna_number_thresh,
                          veto_number_thresh,
                          core_roles_array,
                          veto_roles_array)
            setvalue(snapbrd,"delay_trigger","cr_registers.xlsx",bufferwait)
        except:
                if attempt<tries-1:
                    continue
                else:
                    raise
        break
            

## Run Rate Monitor and Detector

In [ ]:
%%capture output

starttimestring='2024-08-13T06:00:00'
duration=10*60*60 #in seconds
waittime=30 #time between reading monitor

tstart=Time(starttimestring,format='isot',scale='utc')
yet=(time.time()>tstart.unix)
while not yet:
    yet=(time.time()>tstart.unix)
    time.sleep(1)


#When it's time to observe
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",1) for snapbrd in snapbrds] 

finish=tstart.unix+duration

core_triggers=np.zeros((11,int(duration/waittime)))
veto_triggers=np.zeros((11,int(duration/waittime)))
core_cycles=np.zeros((11,int(duration/waittime)))
veto_cycles=np.zeros((11,int(duration/waittime)))
successful_triggers=np.zeros((11,int(duration/waittime)))
readout_dones=np.zeros((11,int(duration/waittime)))


i=0
tries=10

time.sleep(waittime)
while time.time()<finish:
    for b in [0,1,2,3,4,5,6,7,8,9,10]:
        for attempt in range(tries):
            try:
                snapbrd=snapbrds[b]
                veto_coincidence,core_coincidence=snapbrd.read_list_from_ram("cosmic_ray_core_and_veto_rate",2,'u4')
                veto_total_cycles,core_total_cycles=snapbrd.read_list_from_ram("cosmic_ray_core_n_veto_tot_clk_cycles",2,'u4')
                readout_done,successful_trigger=snapbrd.read_list_from_ram("cosmic_ray_core_n_veto_tot_clk_cycles1",2,'u4')
                core_triggers[b,i]=core_coincidence            
                veto_triggers[b,i]=veto_coincidence
                core_cycles[b,i]=core_total_cycles
                veto_cycles[b,i]=veto_total_cycles
                successful_triggers[b,i]=successful_trigger
                readout_dones[b,i]=readout_done
            except:
                if attempt<tries-1:
                    continue
                else:
                    raise
            break
            
    time.sleep(waittime)
    i+=1

#turn trigger off
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]


#save results
np.save(fname_prefix+'core',np.asarray(core_triggers))
np.save(fname_prefix+'veto',np.asarray(veto_triggers))
np.save(fname_prefix+'vetocycles',np.asarray(veto_cycles))
np.save(fname_prefix+'corecycles',np.asarray(core_cycles))
np.save(fname_prefix+'successfultrigger',np.asarray(successful_triggers))
np.save(fname_prefix+'readoutdone',np.asarray(readout_dones))


In [ ]:
output.show()

In [152]:
print(i)

0


## Troubleshoot monitor

In [137]:
print(np.min(core_triggers),np.max(core_triggers))
print(np.min(core_coincidence),np.max(core_coincidence))
print(np.min(veto_coincidence),np.max(veto_coincidence))


0.0 0.0
409 409
0 0


## Troubleshoot ethernet-- run this with f-engine polling turned off

In [252]:
casperbrds = [casperfpga.CasperFpga(brdname, transport=casperfpga.TapcpTransport) for brdname in brdnames]
brdnames=['snap01','snap02','snap03','snap04','snap05','snap06','snap07','snap08','snap09','snap10','snap11']

In [183]:
#disable ethernet and make sure trigger settings are reset for all
for p in range(11):
    casperbrd=casperbrds[p]
    setup_coincidencer(casperbrd,0,0,0,0,0,0,core_roles_array,veto_roles_array)

    setvalue(casperbrd,'eth_enable','cr_registers.xlsx',1) 


In [86]:
i=3


computer='gpu09' # 'lwacr' or 'gpu09'
packetwait=100 #clock cycles between packets
wait_after_readout=200 #time [clock cycles] to wait for other boards to finish readout

casperbrd=casperbrds[i]
brdname=brdnames[i]
#prepare to send a snapshot
setup_data_source(casperbrd,"adc") #set the data source to ADC
setup_ethernet(brdname,casperbrd,fpgfile,computer,packetwait) #set up the ethernet packetizer and ethernet block
setvalue(casperbrd,'wait_after_readout','cr_registers.xlsx',wait_after_readout) #set the wait time after readout before new internally-generated triggers are accepted
setvalue(casperbrd,'brd_id','cr_registers.xlsx',int(brdname[-2:])) #set the board id
setvalue(casperbrd,'eth_enable','cr_registers.xlsx',1)  #enable ethernet

#send trigger
software_trigger(snapbrds[i],1)


Data source set to adc
------------------------
snap04:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:04:04:04
Gateway:  0.0.0.1
IP:  10.41.0.204
Fabric port: 
11111
Fabric interface is currently: Enabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips: []
ARP Table: 
IP: 10.41.0. 25: MAC: 04:3F:72:DF:C1:F0
Setting wait between packets to 100
Packetizer formed 64 packets
Board successfully returned to listening state after readout
Readout took 137669 clock cycles
Packetizer never overflowed
Ethernet block almost-full counter incremented by 0


In [180]:
#send a trigger and then turn off the board's ethernet
i=5
setvalue(casperbrds[i],'eth_enable','cr_registers.xlsx',1)  #enable ethernet
software_trigger(snapbrds[i],1)
setvalue(casperbrds[i],'eth_enable','cr_registers.xlsx',0)  #disable ethernet

for p in range(11):
    casperbrd=casperbrds[p]
    print(p,getvalue(casperbrd,'eth_enable','cr_registers.xlsx'))

Packetizer formed 64 packets
Board successfully returned to listening state after readout
Readout took 137669 clock cycles
Packetizer never overflowed
Ethernet block almost-full counter incremented by 0
0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0


In [259]:
#enable or disable ethernet for all
for p in range(11):
    casperbrd=casperbrds[p]
    setvalue(casperbrd,'eth_enable','cr_registers.xlsx',0) 
    print(getvalue(casperbrd,'eth_enable','cr_registers.xlsx'))

0
0
0
0
0
0
0
0
0
0
0


In [257]:
#send a trigger to one board
software_trigger(snapbrds[10],1)


AssertionError: board didn't reset to listen

## Scratch

In [251]:
[reset_to_listen(snapbrd) for snapbrd in snapbrds]

[print(getvalue(snapbrd,"readout_state_value","cr_registers.xlsx")) for snapbrd in snapbrds]


1
1
1
1
1
1
1
1
1
1
1


[None, None, None, None, None, None, None, None, None, None, None]

In [253]:
[reset_to_listen(snapbrd) for snapbrd in casperbrds]

[print(getvalue(snapbrd,"readout_state_value","cr_registers.xlsx")) for snapbrd in casperbrds]


1
1
1
1
1
1
1
1
1
1
1


[None, None, None, None, None, None, None, None, None, None, None]

In [228]:
setvalue(snapbrds[0],"send_trigger","cr_registers.xlsx",0)
setvalue(snapbrds[0],"send_trigger","cr_registers.xlsx",1)

In [233]:
[print(getvalue(snapbrd,"readout_state_value","cr_registers.xlsx")) for snapbrd in snapbrds]


1
1
1
1
1
1
1
1
1
1
1


[None, None, None, None, None, None, None, None, None, None, None]

In [ ]:
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]

#send trigger to just one board and all will send packets
software_trigger(snapbrds[1],1)

In [ ]:
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]


In [235]:
brd=snapbrds[0]
reset_to_listen(brd)
val=getvalue(brd,"readout_state_value","cr_registers.xlsx")
assert val==0, "board didn't reset to listen"

AssertionError: board didn't reset to listen

In [236]:
getvalue(brd,"readout_state_value","cr_registers.xlsx")

1

In [237]:
assert val==0, "board didn't reset to listen"

AssertionError: board didn't reset to listen

In [250]:
[print(getvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx")) for snapbrd in snapbrds]


0
0
0
0
0
0
0
0
0
0
0


[None, None, None, None, None, None, None, None, None, None, None]

In [248]:
setvalue(brd,'reset_to_listen','cr_registers.xlsx',1)
print(getvalue(brd,"reset_to_listen","cr_registers.xlsx"))
print(getvalue(brd,"readout_state_value","cr_registers.xlsx"))

1
1


In [243]:
getvalue(brd,'reset_to_listen','cr_registers.xlsx')
getvalue(brd,"readout_state_value","cr_registers.xlsx")

1

In [256]:
brd2=casperbrds[2]  #not going through etcd
setvalue(brd2,'reset_to_listen','cr_registers.xlsx',1)
print(getvalue(brd2,"reset_to_listen","cr_registers.xlsx"))
print(getvalue(brd2,"readout_state_value","cr_registers.xlsx"))

1
1


In [ ]:
print(np.max(core_triggers))
print(core_triggers.shape)
print(8*60*60/30)
print(i)
print(int(duration/waittime))
print(i,(i*waittime)/(60*60))

In [ ]:
starttimestring='2024-07-03T23:10:30'
duration=10 #in seconds
waittime=1 #time between reading monitor

tstart=Time(starttimestring,format='isot',scale='utc')
yet=(time.time()>tstart.unix)
while not yet:
    yet=(time.time()>tstart.unix)
    time.sleep(1)
finish=tstart.unix+duration
while time.time()<finish:
    print('working')
    time.sleep(1)

In [ ]:
import argparse
import numpy as np
import casperfpga
import time
from lwa_f import snap2_fengine
from lwa_f import blocks
from cr_functions import *

#parse arguments
parser=argparse.ArgumentParser(description='Run cosmic ray trigger on muon detector only')
parser.add_argument('threshold',type=int,help="Power threshold for muon detector.")
parser.add_argument('duration', type=str, help= "Number of seconds to run detector.")

args=parser.parse_args()

## main parameters to adjust
trigger_power_thresh=args.threshold
duration = args.duration

#turn off the trigger for all the snap boards
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]

# which snap board and input correspond to the muon detector
snapbrd=snapbrds[3]  
ant_index=

#set up coincidencer
veto_power_thresh=0
trigger_window=10 #a small positive number
veto_window=0 #not using
antenna_number_thresh=0  #trigger on only one antenna
veto_number_thresh=64 #turn off
veto_roles_array=np.zeros(64)
core_roles_array=np.zeros(64)
core_roles_array[ant_index]=1  #set only the muon detector to participate
bufferwait=2000

#configure trigger
tries=15
for attempt in range(tries):
    try:
        setup_coincidencer(snapbrd,trigger_power_thresh,
                           veto_power_thresh,
                          trigger_window,
                          veto_window,
                          antenna_number_thresh,
                          veto_number_thresh,
                          core_roles_array,
                          veto_roles_array)
        setvalue(snapbrd,"delay_trigger","cr_registers.xlsx",bufferwait)
        except:
                if attempt<tries-1:
                    continue
                else:
                    raise
        break
            
#run the trigger for the desired length of time and then turn it off
setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",1)
time.sleep(duration)
setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0)

In [261]:
for s in range(11):
    snapbrd=snapbrds[s]
    veto_coincidence,core_coincidence=snapbrd.read_list_from_ram("cosmic_ray_core_and_veto_rate",2,'u4')
    veto_total_cycles,core_total_cycles=snapbrd.read_list_from_ram("cosmic_ray_core_n_veto_tot_clk_cycles",2,'u4')
    readout_done,successful_trigger=snapbrd.read_list_from_ram("cosmic_ray_core_n_veto_tot_clk_cycles1",2,'u4')
    print(core_coincidence,veto_coincidence,core_total_cycles,veto_total_cycles,successful_trigger,readout_done)

0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0


In [265]:
[setvalue(brd,'eth_enable','cr_registers.xlsx',0) for brd in casperbrds]


[None, None, None, None, None, None, None, None, None, None, None]

In [266]:
[getvalue(brd,'eth_enable','cr_registers.xlsx') for brd in casperbrds]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
rigger_count

In [280]:
[print(getvalue(brd,'count_triggers','cr_registers.xlsx')) for brd in casperbrds]
time.sleep(5)
print('')
[print(getvalue(brd,'count_triggers','cr_registers.xlsx')) for brd in casperbrds]


6872192
6867217
6867146
6867108
1761046230
6913902
6872363
6872532
6872730
6873145
6907265

6879586
6874901
6874838
6874807
1828562619
6922056
6880524
6880700
6880912
6881327
6915457


[None, None, None, None, None, None, None, None, None, None, None]

In [281]:
[print(getvalue(brd,'count_snapshots','cr_registers.xlsx')) for brd in casperbrds]
time.sleep(5)
print('')
[print(getvalue(brd,'count_snapshots','cr_registers.xlsx')) for brd in casperbrds]


6903531
6903476
6903405
6903403
6950195
6943461
6908660
6908722
6908999
6909538
6910465

6911368
6911298
6911225
6911227
6958012
6951280
6916478
6916541
6916816
6916921
6917839


[None, None, None, None, None, None, None, None, None, None, None]

In [279]:
[print(getvalue(brd,'readout_state_value','cr_registers.xlsx')) for brd in casperbrds]


1
1
1
1
1
1
1
1
1
1
1


[None, None, None, None, None, None, None, None, None, None, None]

In [ ]:
readout_state
trigger_count
trig_timer
snapshot_counter

In [278]:
for p in range(11):    
    casperbrd=casperbrds[p]
    setvalue(casperbrd,'eth_enable','cr_registers.xlsx',1) 
    print(getvalue(casperbrd,'eth_enable','cr_registers.xlsx'))

1
1
1
1
1
1
1
1
1
1
1


In [284]:
[print(getvalue(brd,'eof','cr_registers.xlsx')) for brd in casperbrds]


1001270792
476432945
476427503
476427302
1004247000
479003879
476783051
476788165
476808655
476816659
476882820


[None, None, None, None, None, None, None, None, None, None, None]